# 🔓 RSA Attack Level 3 – Reuse of Public Key with Small e

Serangan ini terjadi ketika **pesan yang sama** dienkripsi ke beberapa penerima berbeda menggunakan eksponen kecil (e = 3). Jika `m^e < n1*n2*n3`, attacker bisa menggabungkan ciphertext dan mengambil akar ke-e untuk mendapatkan plaintext **tanpa private key**.

📌 Teknik: Håstad’s Broadcast Attack + Chinese Remainder Theorem

✅ Syarat:
- e = 3
- 3 modulus berbeda (coprime)
- Pesan yang sama (tanpa padding)

🎯 Tujuan: Mendapatkan plaintext dari ciphertext gabungan.


In [7]:
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes
from functools import reduce
import gmpy2

# Fungsi Chinese Remainder Theorem (CRT)
def crt(c_list, n_list):
    N = reduce(lambda a, b: a * b, n_list)
    total = 0
    for c_i, n_i in zip(c_list, n_list):
        m_i = N // n_i
        inv = gmpy2.invert(m_i, n_i)
        total += c_i * m_i * inv
    return total % N


In [8]:
# Plaintext yang akan dienkripsi (bisa disesuaikan)
plaintext = b"Daffa Dimas Nova"
m = bytes_to_long(plaintext)  # Konversi ke integer
e = 3  # Eksponen publik kecil

# Fungsi untuk menghasilkan modulus RSA (n = p * q)
def generate_modulus(bits=512):
    return getPrime(bits) * getPrime(bits)


In [9]:
# Generate 3 pasang modulus dan ciphertext
n_list = [generate_modulus() for _ in range(3)]
c_list = [pow(m, e, n) for n in n_list]

# Gabungkan ciphertext menggunakan CRT
c_combined = crt(c_list, n_list)

# Ambil akar pangkat 3 dari hasil CRT
recovered_m, exact = gmpy2.iroot(c_combined, e)
recovered_text = long_to_bytes(recovered_m)


In [10]:
print("===== Attack 3: Reuse Public Key with Small e (gmpy2) =====")
for i in range(3):
    print(f"Modulus n{i+1} = {n_list[i]}")
    print(f"Ciphertext c{i+1} = {c_list[i]}")
print("\nHasil gabungan CRT:", c_combined)
print(f"Akar pangkat 3 (recovered m): {recovered_m}")
print(f"Teks hasil dekripsi: '{recovered_text.decode()}'")
print(f"Berhasil? {recovered_text == plaintext}")


===== Attack 3: Reuse Public Key with Small e (gmpy2) =====
Modulus n1 = 156823168862889792249861983286094439097340925969493873776640113312941932681762653315008785151873218266239810994370110517332626984209872687098323806534470361551476727801316592598697394448932751046024271804204136967586743151808700444142767947111243597314843204536408893769440214650625860403255753137203707848303
Ciphertext c1 = 750921708733871343535259110877586015283603368964208968626728870807915851311530807651019564503018616501443749334817
Modulus n2 = 65339284529365972165163015090785401167486069555500292064635472647104229898366351099394133644019516909307117444920488829180933597650369702225119612037412332639678292434602255757247638214028131838868013500682006293626521121512870954110903790637731916613300644907210322372182056176009782231426665613963117066873
Ciphertext c2 = 750921708733871343535259110877586015283603368964208968626728870807915851311530807651019564503018616501443749334817
Modulus n3 = 88161668227429015042